<a href="https://colab.research.google.com/github/RyEggGit/FCC-ML/blob/main/fcc_sms_text_classification_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


print(tf.__version__)

In [ ]:
# My imports 
from keras.preprocessing import sequence, text
from keras import Sequential, layers
import math
from collections import Counter
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM, Bidirectional


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"

test_file_path = "valid-data.tsv"

# Format Data from CSV
- 

In [ ]:
# Read .tsv
trainDF = pd.read_csv(train_file_path, delimiter="\t",encoding='UTF-8',names=["label", "message"])
testDF = pd.read_csv(test_file_path, delimiter="\t",encoding='UTF-8',names=["label", "message"])

# Preprocessing
- Combine Data sets
- Pop the Y category from X
- Determine the length of the sequnece from average

In [ ]:
print(trainDF.shape)
print(testDF.shape)
print(len(testDF)/len(trainDF))

In [ ]:
#----Pre Processing----
trainDF = trainDF.append(testDF, ignore_index=True)
#Covert cat-> num
trainDF['label'] = trainDF['label'].astype("category").cat.codes
# Seperate X and Y 

train_Y = trainDF.pop('label')
train_X = trainDF
# Get Sequence Length
SEQUNECE_LENGTH = math.floor(trainDF['message'].apply(len).mean()) - 20 
print(SEQUNECE_LENGTH)
print(trainDF.shape)

# Anyalize Data
- Alot more "ham"s than "spam"s so undersample inorder to have better data
- Tokenize/ pad data for model to use

In [ ]:
print(train_Y.value_counts())

In [ ]:
#Undersample Approach
print(len(train_X),len(train_Y))
percent_to_remove = round(1-train_Y.value_counts()[1]/train_Y.value_counts()[0], 2)
remove_vals = train_Y[train_Y == 0].sample(frac = percent_to_remove)
for key in remove_vals.keys():
  train_X = train_X.drop(key)
  train_Y = train_Y.drop(key)
print(len(train_X),len(train_Y))


In [ ]:
print(train_Y.value_counts())

In [ ]:
# Turn strings to list of ints
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(train_X['message'].tolist())
train_X = tokenizer.texts_to_sequences(train_X['message'].tolist())

vocab_size = tokenizer.document_count

In [ ]:
# Pad messages
train_X = tf.keras.preprocessing.sequence.pad_sequences(train_X, SEQUNECE_LENGTH)


#Create Model


In [ ]:
#Dense model architecture
model = Sequential()
model.add(Embedding(5000, 32, input_length=SEQUNECE_LENGTH))
model.add(LSTM(32, activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy'])
model.summary()

In [ ]:
# Train Model
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(train_X, train_Y,epochs=20,verbose=True,validation_split= 0.2, callbacks=[early_stop])

#Analyze Loss/Accuraccy of train vs test Data

In [ ]:
# Read as a dataframe 
metrics = pd.DataFrame(history.history)
# Rename column
metrics.rename(columns = {'loss': 'Training_Loss', 'accuracy': 'Training_Accuracy', 'val_loss': 'Validation_Loss', 'val_accuracy': 'Validation_Accuracy'}, inplace = True)
def plot_graphs1(var1, var2, string):
    metrics[[var1, var2]].plot()
    plt.title('Training and Validation ' + string)
    plt.xlabel ('Number of epochs')
    plt.ylabel(string)
    plt.legend([var1, var2])
plot_graphs1('Training_Loss', 'Validation_Loss', 'loss')


In [ ]:
plot_graphs1('Training_Accuracy', 'Validation_Accuracy', 'accuracy')

#Format Output In order to Make Predictions

In [ ]:
def encode_text(uncode_text):
  uncode_text = [uncode_text]
  tokenizer.fit_on_texts(uncode_text)
  
  code_text = tokenizer.texts_to_sequences(uncode_text)
  code_text = keras.preprocessing.sequence.pad_sequences(code_text, SEQUNECE_LENGTH)
  return code_text

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  
  pred_text = encode_text(pred_text)
  pred = np.zeros((1, SEQUNECE_LENGTH))
  pred[0] = pred_text
  prediction = model.predict(pred)
  if (prediction[0][0] >= 0.5):
      prediction= np.append(prediction[0],['spam'])
  else:
      prediction = np.append(prediction[0],['ham'])
  return prediction
  

pred_text = "our new mobile video service is live. just install on your phone to start watching."

prediction = predict_message(pred_text)
print(prediction)

#Test

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      print(prediction[0],msg)
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
